In [1]:
orcid = '0000-0002-0566-0759' # Fill your orcid here

In [2]:
import requests

We use the `/works` api to list all works related to the orcid. This gives a summary of all works, so citation information is not included. We collect the `put-code` of all works to retrieve the citation information later.

In [7]:
response = requests.get('https://pub.orcid.org/v3.0/{}/works'.format(orcid),
                        headers={"Accept": "application/orcid+json" })
record = response.json()

In [38]:
put_codes = []
for work in record['group']:
    put_code = work['work-summary'][0]['put-code']
    put_codes.append(put_code)
put_code = put_codes[0]

In [34]:
put_codes = []
for work in record['group']:
    for summary in work['work-summary']:
        put_codes.append(summary['put-code'])


We use the `/<orcid>/work/<put-code>` endpoint to retrieve the citation information for each record.

In [39]:
citations = []
for put_code in put_codes:
    response = requests.get('https://pub.orcid.org/v3.0/{}/work/{}'.format(orcid, put_code),
                            headers={"Accept": "application/orcid+json" })
    work = response.json()
    try:
        if work['citation'] is not None:
            citations.append(work['citation']['citation-value'])
    except KeyError:
        pass

In [40]:
citations

['@article{\nPigani2022delay,\nauthor = {Emanuele Pigani  and Damiano Sgarbossa  and Samir Suweis  and Amos Maritan  and Sandro Azaele },\ntitle = {Delay effects on the stability of large ecosystems},\njournal = {Proceedings of the National Academy of Sciences},\nvolume = {119},\nnumber = {45},\npages = {e2211449119},\nyear = {2022},\ndoi = {10.1073/pnas.2211449119},\nURL = {https://www.pnas.org/doi/abs/10.1073/pnas.2211449119},\neprint = {https://www.pnas.org/doi/pdf/10.1073/pnas.2211449119},\nabstract = {Understanding how communities emerge from a large number of interacting entities is a long-standing question in several fields. In ecosystems with randomly coupled species, a delayed dynamics seemed to play a minor role in characterizing the stability close to equilibrium. Here, we study the effects on large ecosystems of species’ interactions that are random as well as delayed. We find that near equilibrium, delayed self-interactions greatly modify the eigenspectrum distribution as 

In [27]:
citations

['@article{\ndoi:10.1073/pnas.2211449119,\nauthor = {Emanuele Pigani  and Damiano Sgarbossa  and Samir Suweis  and Amos Maritan  and Sandro Azaele },\ntitle = {Delay effects on the stability of large ecosystems},\njournal = {Proceedings of the National Academy of Sciences},\nvolume = {119},\nnumber = {45},\npages = {e2211449119},\nyear = {2022},\ndoi = {10.1073/pnas.2211449119},\nURL = {https://www.pnas.org/doi/abs/10.1073/pnas.2211449119},\neprint = {https://www.pnas.org/doi/pdf/10.1073/pnas.2211449119},\nabstract = {Understanding how communities emerge from a large number of interacting entities is a long-standing question in several fields. In ecosystems with randomly coupled species, a delayed dynamics seemed to play a minor role in characterizing the stability close to equilibrium. Here, we study the effects on large ecosystems of species’ interactions that are random as well as delayed. We find that near equilibrium, delayed self-interactions greatly modify the eigenspectrum dist

In [28]:
with open('output.bib', 'w') as bibfile:
    for citation in citations:
        bibfile.write(citation)
        bibfile.write('\n')

In [42]:
import requests
from time import sleep
import re

ORCID = "0000-0002-0566-0759"
BASE = "https://pub.orcid.org/v3.0"
HDRS = {"Accept": "application/orcid+json"}

def safed(d, *keys, default=None):
    """Nested dict getter: safed(x,'a','b','c',default=None)"""
    for k in keys:
        if isinstance(d, dict) and k in d:
            d = d[k]
        else:
            return default
    return d

def slugify(s, n=5):
    tokens = re.findall(r"[A-Za-z0-9]+", s or "")
    return "".join(tokens[:n])

# 1) Prendi tutti i put-code (tutte le work-summary, non solo [0])
resp = requests.get(f"{BASE}/{ORCID}/works", headers=HDRS)
resp.raise_for_status()
record = resp.json()

put_codes = []
titles_index = {}  # per debug/chiavi
for grp in record.get("group", []):
    for summary in grp.get("work-summary", []):
        pc = summary.get("put-code")
        if pc is not None:
            put_codes.append(pc)
            titles_index[pc] = safed(summary, "title", "title", "value", default="")

# de-duplica mantenendo l’ordine
seen = set()
put_codes = [pc for pc in put_codes if not (pc in seen or seen.add(pc))]

print(f"Found {len(put_codes)} works")

# 2) Scarica ogni work completa e ricava una citazione (o fallback BibTeX)
citations = []
for pc in put_codes:
    r = requests.get(f"{BASE}/{ORCID}/work/{pc}", headers=HDRS)
    if r.status_code != 200:
        continue
    work = r.json()

    # Se ORCID ha già la citazione formattata, usala
    cit = safed(work, "citation", "citation-value")
    if cit:
        citations.append(cit.strip())
        sleep(0.2)
        continue

    # --- Fallback BibTeX minimale ---
    title = safed(work, "title", "title", "value", default=titles_index.get(pc, ""))
    journal = safed(work, "journal-title", "value", default="")
    year = safed(work, "publication-date", "year", "value", default="")
    # autori
    contribs = safed(work, "contributors", "contributor", default=[]) or []
    authors = []
    for c in contribs:
        name = safed(c, "credit-name", "value")
        if not name:
            # prova dato strutturato
            given = safed(c, "contributor-name", "given-names", "value", default="")
            family = safed(c, "contributor-name", "family-name", "value", default="")
            name = (given + " " + family).strip()
        if name:
            authors.append(name)
    authors_str = " and ".join(authors) if authors else ""

    # DOI se presente
    doi = ""
    for ext in safed(work, "external-ids", "external-id", default=[]) or []:
        if safed(ext, "external-id-type", default="").lower() == "doi":
            doi = safed(ext, "external-id-value", default="")
            break

    # chiave BibTeX
    first_author_key = slugify(authors[0].split()[-1] if authors else "noauthor", 1)
    key = f"{first_author_key}{year}{slugify(title,1)}"

    # tipo (article default)
    entry_type = "article"
    wtype = safed(work, "type", default="").lower()
    if "book" in wtype: entry_type = "book"
    elif "conference" in wtype or "proceeding" in wtype: entry_type = "inproceedings"

    # costruisci BibTeX semplice
    lines = [f"@{entry_type}{{{key},"]
    if title:   lines.append(f"  title = {{{title}}},")
    if authors_str: lines.append(f"  author = {{{authors_str}}},")
    if journal and entry_type == "article": lines.append(f"  journal = {{{journal}}},")
    if year:    lines.append(f"  year = {{{year}}},")
    if doi:     lines.append(f"  doi = {{{doi}}},")
    lines[-1] = lines[-1].rstrip(",")  # niente virgola finale
    lines.append("}")
    citations.append("\n".join(lines))
    sleep(0.2)

# 3) Salva tutto in .bib
with open("output.bib", "w", encoding="utf-8") as f:
    for c in citations:
        f.write(c.strip() + "\n\n")

print("Written output.bib with", len(citations), "entries")


Found 8 works
Written output.bib with 8 entries


In [22]:
import requests, json, re, pathlib, time
ORCID = "0000-0002-0566-0759"
BASE = "https://pub.orcid.org/v3.0"
HDRS = {"Accept": "application/orcid+json"}

def g(d,*ks,default=None):
    for k in ks:
        if isinstance(d,dict) and k in d: d=d[k]
        else: return default
    return d

# 1) Elenco "tutti i works pubblici" visti dall’endpoint /works
r = requests.get(f"{BASE}/{ORCID}/works", headers=HDRS); r.raise_for_status()
record = r.json()

summaries = []
for grp in record.get("group", []):
    for s in grp.get("work-summary", []):
        summaries.append({
            "put_code": s.get("put-code"),
            "title": g(s,"title","title","value",""),
            "visibility": s.get("visibility"),
            "type": s.get("type"),
            "source": g(s,"source","source-name","value",""),
        })

print(f"[ORCID /works] lavori pubblici trovati: {len(summaries)}")
for i,x in enumerate(summaries,1):
    print(f"{i:02d}. [{x['visibility']}] {x['title']}  (put:{x['put_code']}, type:{x['type']}, src:{x['source']})")

# 2) Provo a scaricare ogni /work/<put_code> e verifico chi fallisce o non ha citazione
missing = []
no_citation = []
ok = []

for x in summaries:
    pc = x["put_code"]
    if pc is None: 
        missing.append((x,"no put-code in summary"))
        continue
    rr = requests.get(f"{BASE}/{ORCID}/work/{pc}", headers=HDRS)
    if rr.status_code != 200:
        missing.append((x,f"HTTP {rr.status_code}"))
        continue
    w = rr.json()
    cit = g(w, "citation", "citation-value")
    if not cit:
        no_citation.append(x)
    else:
        ok.append(x)
    time.sleep(0.2)  # evito rate limit

print("\n---- RIEPILOGO ----")
print("OK con citazione:", len(ok))
print("Senza citazione (verrà creato BibTeX di fallback):", len(no_citation))
print("Non scaricabili:", len(missing))
for x,why in missing:
    print(f" - {x['title']}  (put:{x['put_code']}) -> {why}")

# 3) Se vuoi vedere subito quali TITOLI NON sono finiti nell'output.bib prodotto prima:
bib = pathlib.Path("output.bib").read_text(encoding="utf-8") if pathlib.Path("output.bib").exists() else ""
present = []
for x in summaries:
    t = re.escape((x["title"] or "")[:40])
    if re.search(t, bib, flags=re.I):
        present.append(x["title"])
absent = [x["title"] for x in summaries if x["title"] not in present]
print("\nTitoli non trovati in output.bib:", len(absent))
for t in absent:
    print(" -", t)


[ORCID /works] lavori pubblici trovati: 9
01. [public] None  (put:181179951, type:preprint, src:None)
02. [public] None  (put:154988809, type:journal-article, src:None)
03. [public] None  (put:121985851, type:journal-article, src:None)
04. [public] None  (put:113427012, type:journal-article, src:None)
05. [public] None  (put:102502489, type:journal-article, src:None)
06. [public] None  (put:90495761, type:journal-article, src:None)
07. [public] None  (put:87322802, type:journal-article, src:None)
08. [public] None  (put:77654936, type:preprint, src:None)
09. [public] None  (put:77225035, type:other, src:None)

---- RIEPILOGO ----
OK con citazione: 5
Senza citazione (verrà creato BibTeX di fallback): 4
Non scaricabili: 0

Titoli non trovati in output.bib: 0
